# <center> RAG для нормативно правовых актов </center>

In [37]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

from datasets import Dataset 

from llamaapi import LlamaAPI
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

from pymilvus import MilvusClient

from ragas import evaluate
from ragas.metrics import (
    answer_сorrectness,
    faithfulness,
    context_recall,
    context_precision,
)

LLAMA_API_TOKEN = 'LA-9c513f00d473403cb377f52b84b1fc3fa0826b643f974c26bdace543403810e4'

ImportError: cannot import name 'answer_сorrectness' from 'ragas.metrics' (/Users/danilovsnnv/Work/venvs/python3.10/lib/python3.10/site-packages/ragas/metrics/__init__.py)

### Функции для подготовки данных для хранилища

Функции для создания эмбеддингов. Надо заняться тем, чтобы сделать на сервере эмбеддинг каждого акта при помощи mistral e5

In [2]:
# For embedding model multilingual-e5-base

def average_pool(
    last_hidden_states: Tensor,
     attention_mask: Tensor
) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def make_embedding(input_texts, model, tokenizer):
    batch_dict = tokenizer(
        input_texts, max_length=512, 
        padding=True, truncation=True, 
        return_tensors='pt'
    )
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.tolist()

В milvus можно вставлять данные как список словарей. В каждом словаре в поле "vector" лежит эмбединг, потом по этому полю будет идти поиск. Остальные поля это мета данные. Тут я пока не подогла поля под нашу задачу

In [3]:
def converting_data(embeds, summurys, original_texts, id_begin=0):
    data = []

    for i, (embed, summury, original_text) in enumerate(zip(embeds, summurys, original_texts)):
        data.append({
            'id': id_begin + i, 
            'vector': embed, 
            'summary': summury, 
            'original_text': original_text,  # TODO: add meta data
        })
    
    return data

In [22]:
system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать бонус.
"""

question = """ 
    Ты разговариваешь только на русском языке. Ты специалист по юристпруденции. 
    Тебе на вход приходит нормативно правовой акт. Суммаризируй его, найди основную суть документа. 
    Твой ответ должен быть кратким, в ответе просто напиши саммари без лишних вводных фраз. Особое внимание удели числам, процентам, датам и внесенным изменениям.
    Нормативно-правовой акт:
    {context}
"""

question_json = """ 
    Ты разговариваешь только на русском языке. Ты специалист по юристпруденции. 
    Тебе на вход приходит нормативно правовой акт. 
    На выходе ты должен отдать только JSON в следующем формате:
    
    {{'Название акта': 'Здесь помести только название нормативно правового акта', 
    'Регистрационный номер': ' Здесь помести только регистрационный номер НПА',
    'Дата принятия': 'Здесь Дата принятия (когда нормативный акт был официально принят или издан)',  
    'Дата вступления': 'Дата вступления в силу (по общему правилу через 10 календарных дней после официальной публикации, если в самом акте не сказано иное; есть исключения для налоговых и бюджетных нпа - они со следующего налогового периода или периода бюджетного планирования вступают в силу)',
    'Дата опубликования': 'Дата, когда НПА был опубликован в официальном источнике, чтобы  отследить его юридическую силу',
    'Орган': ' Здесь орган, издавший акт',
    'Тип документа': 'Здесь тип документа (например закон, указ, постановление, приказ)',
    'Сфера регулирования': 'Здесь Сфера регулирования: (сфера общественных отношений, на которую распространяется действие НПА, например, трудовые отношения, налогообложение, бюджетная политика)',
    'Ключнвые слова': 'Перечисли слова по которым можно найти этот документ'
    }}

    Ны выходе должен быть только json размера 7.
    Ответ должен быть обязатьльно только в формате json.
    Нормативно-правовой акт:
    {context}
"""

In [38]:
def connect_to_llama_by_api(llama_token):
    llama = LlamaAPI(llama_token)
    return llama

def get_answer_by_api(context, llm, system_prompt, question, max_tokens=4096, temperature=0.1, quary=''):
    api_request_json = {
        'model': 'llama3.1-8b',
        'max_tokens': max_tokens,
        'temperature': temperature,
        "messages": [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question.format(context=context, quary=quary)}
        ]
    }
    response = llm.run(api_request_json)
    return response.json()['choices'][0]['message']['content']
    

In [39]:
def make_summary(docs, llm, system_prompt, question, get_answer_func=get_answer_by_api):
    summarys = []
    for doc in tqdm(docs):
        summarys.append(get_answer_func(doc, llm, system_prompt, question))
    return summarys

In [87]:
summarys

['В постановлении Губернатора Ханты-Мансийского автономного округа-Югры от 28.12.2017 № 139 внесены изменения в Инструкцию по делопроизводству в государственных органах Ханты-Мансийского автономного округа-Югры и исполнительных органах государственной власти Ханты-Мансийского автономного округа-Югры.\n\nИзменения вступают в силу с 1 июля 2018 года.\n\nОсновные изменения:\n\n- В разделе I заменены ссылки на ГОСТ Р 6.30-2003 на ГОСТ Р 7.0.97-2016.\n- В разделе III внесены изменения в пункты 3.7, 3.15, 3.16, 3.18, 3.19, 3.20, 3.24, 3.25.\n- В разделе IV внесены изменения в пункты 4.21, 4.22, 4.24, 4.56.1.\n- В разделе VII внесены изменения в подпункт 7.4.6.\n- В разделе X заменены ссылки на ГОСТ Р 6.30-2003 на ГОСТ Р 7.0.97-2016.',
 'В постановлении Правительства Ханты-Мансийского автономного округа - Югры от 01.10.2021 № 406-п внесены изменения в постановление Правительства Ханты-Мансийского автономного округа - Югры от 22 августа 2014 года N 306-п "О нормах питания получателей социальны

### Подготовка данных для хранилища

In [40]:
data_path = '../../../data/rag_npa/hmao_npa.txt'  # path to dataset

with open(data_path, encoding='utf8') as file:
    txt = file.read()

list_docs = txt.split('\n')
list_docs = [doc for doc in list_docs if doc]

In [41]:
new_docs = []
numbers = []
for i in range(len(list_docs)):
    ids = list_docs[i].find('¦')
    if ids == -1:
        ids = list_docs[i].find('|')
    if ids == -1:
        new_docs.append(list_docs[i])
    else:
        new_docs.append(list_docs[i][:ids])
        numbers.append(i)

In [27]:
max_len = np.max([len(doc.split()) for doc in new_docs]) # if len(doc.split()) != max_len])
for doc in new_docs:
    if len(doc.split()) == max_len:
        bad_doc = doc

In [42]:
llama = connect_to_llama_by_api(LLAMA_API_TOKEN)

In [43]:
response = get_answer_by_api(bad_doc, llama, system_prompt, question_json)
print(response)
json_response = json.loads(response)

print()
response_2 = get_answer_by_api(bad_doc, llama, system_prompt, question)
print(response_2)

Exception: POST 400 Failed to process your request. Please try again later. If you are processing a function, try using a larger model (70b) for better function formatting.

In [44]:
# Пример json файла
get_answer_by_api(list_docs[0], llama, system_prompt, question_json)

"{'Название акта': 'Постановление Губернатора Ханты-Мансийского автономного округа - Югры от 28 декабря 2017 года N 139',\n 'Регистрационный номер': '139',\n 'Дата принятия': '28 декабря 2017 года',\n 'Дата вступления': '1 июля 2018 года',\n 'Дата опубликования': '28 декабря 2017 года',\n 'Орган': 'Губернатор Ханты-Мансийского автономного округа - Югры',\n 'Тип документа': 'Постановление',\n 'Сфера регулирования': 'Делопроизводство в государственных органах и исполнительных органах государственной власти Ханты-Мансийского автономного округа - Югры',\n 'Ключевые слова': 'делопроизводство, государственные органы, исполнительные органы, Ханты-Мансийский автономный округ - Югра, ГОСТ Р 7.0.97-2016'}"

In [114]:
# создание суммаризации текста
summarys = make_summary(list_docs[:10], llama, system_prompt, question)  # TODO: use full data

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.62s/it]


In [115]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
emb_model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

embeddings = make_embedding(summarys, emb_model, tokenizer)
embedding_dim = len(embeddings[0])
insert_data = converting_data(embeddings, summarys, list_docs)

### Создание коллекции данных

In [116]:
client = MilvusClient(uri="./milvus.db")
# client = MilvusClient(uri="http://localhost:19530", token="root:Milvus")
collection_name = "acts_new"

I0000 00:00:1725966391.071786   41602 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [117]:
client.has_collection(collection_name=collection_name)

False

In [118]:
# удаление коллекции

# if client.has_collection(collection_name=collection_name):
#     client.drop_collection(collection_name=collection_name)

In [119]:
# Создание колекции данных
client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type='COSINE'
)

# Вставка данных в коллекцию
client.insert(collection_name=collection_name, data=insert_data)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'cost': 0}

## Поиск по коллекции

In [120]:
# можно делать запрос на переформулировку

quary = 'Кто явяется пресс-секретарем губернатора ХМАО?'

embedding_quary = make_embedding(quary, emb_model, tokenizer)

### Поиск топ 5 подходящих документов

In [121]:
search_res = client.search(
    collection_name=collection_name,
    data=embedding_quary, 
    limit=5,
    search_params={
        'metric_type': 'COSINE', 
        'params': {}
    }, 
    output_fields=['summury'],
)

In [122]:
retrieved_lines_with_distances = [
    (res['entity']['summury'], res['distance']) for res in search_res[0]
]

for text, res in retrieved_lines_with_distances:
    print(text, '\n')
    print('Близость =', res, '\n\n')

KeyError: 'summury'

### Генерация ответа LLM

In [70]:
question_3 = """ 
    Ты разговариваешь только на русском языке. Ты юрист. Определи чего хочет клиент. Сформулируй его вопрос на юридическом языке.
    Твой ответ должен содержать только сам переформулированный вопрос.
    Вопрос клиента: {context}
"""

llm = connect_to_llama_by_api(LLAMA_API_TOKEN)

quary = get_answer_by_api(quary, llm, system_prompt, question_3, max_tokens=4096, temperature=0.001)
print(quary)

Кто является лицом, ответственным за официальное распространение информации о деятельности губернатора Ханты-Мансийского автономного округа - Югры, в соответствии с действующим законодательством Российской Федерации.


In [71]:
acts_for_answer = '\n\n'.join([res["entity"]["summury"] for res in search_res[0]])

In [73]:
system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать бонус.
"""

question = """ 
    Ты разговариваешь только на русском языке. Ответь на вопрос, опираясь на только нормативно-правовые акты из контектса.
    {quary} 
    
    Нормативные акты: {context}
"""
llm = connect_to_llama_by_api(LLAMA_API_TOKEN)


print(get_answer_by_api(acts_for_answer, llm, system_prompt, question, max_tokens=4096, temperature=0.001, quary=quary))


В соответствии с нормативными актами, указанными в контексте, лицом, ответственным за официальное распространение информации о деятельности губернатора Ханты-Мансийского автономного округа - Югры, является Пресс-секретарь Губернатора.

Это следует из постановления Губернатора Ханты-Мансийского автономного округа-Югры от 17.12.2010 № 236, в котором добавлена должность "Пресс-секретарь Губернатора" в категории "Помощники (советники)" Группы "Ведущие".

Таким образом, Пресс-секретарь Губернатора является лицом, ответственным за официальное распространение информации о деятельности губернатора Ханты-Мансийского автономного округа - Югры.

Бонус: 1


### Оценка метрик при помощи RAGAS

In [34]:
test_set_path = '../../../data/rag_npa/v2_ragas_npa_dataset_firstPart.xlsx'
test_set_df = pd.read_excel(test_set_path)
test_set_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"Каков объем экспорта услуг категории ""Поездки""...","['Увеличение объема экспорта услуг категории ""...","Объем экспорта услуг категории ""Поездки"" в рег...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
1,"Каковы были основные мероприятия, реализуемые ...",['и | |развития | | |Ханты-Мансийского |исполн...,"Основные мероприятия, реализуемые Правительств...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
2,Каков размер финансирования из федерального бю...,['не относящиеся к районам Крайнего Севера и п...,Ответ на данный вопрос отсутствует в контексте.,simple,[{'file_path': '/home/cias/projects/other/raga...,True
3,Какой комплексный центр социального обслуживан...,"['|2.2. |Центр социальной |240 к/мест |7580,0 ...",Комплексный центр социального обслуживания нас...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
4,Какое значение имеет экологическое образование...,['(полного)общего образования и начального про...,Экологическое образование играет важную роль в...,simple,[{'file_path': '/home/cias/projects/other/raga...,True


In [35]:
test_set_df['contexts'] = test_set_df['contexts'].apply(lambda contexts: contexts[2:-2].split("', '"))

In [36]:
test_set_df['answer'] = 'Я не могу ответить на ваш вопрос'  # TODO: generate answers for test set
test_dataset = Dataset.from_pandas(test_set_df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 500
})

In [ ]:
metrics = [
    context_precision,
    faithfulness,
    answer_сorrectness,
    context_recall,
]

result = evaluate(
    test_dataset,
    metrics=metrics,
    llm=model,
    embeddings=emb_model
)